In [79]:
%pip install pandas
%pip install emoji
%pip install datasets
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   -------------------------------------- - 10.2/10.7 MB 49.1 MB/s eta 0:00:01
   ---------------------------------------- 10.7/10.7 MB 44.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.4 MB ? eta -:--:--
   ---------- ----------------------------- 10.2/38.4 MB 53.0 MB/s eta 0:00:01
   ---------------------- ----------------- 21.2/38.4 MB 51.5 MB/s eta 0:00:01
   -------------------------------- ------- 31.5/38.4 MB 49.9 MB/s eta 0:00:01
   ---------------------------------------  38.3/38.4 MB 49.6 MB/s eta 0:00:01
   ---------------------------------------- 38.4/38.4 MB 45.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [80]:
import pandas as pd
import re
import unicodedata
import emoji
from datasets import load_dataset
from sklearn.metrics import f1_score, hamming_loss, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

In [208]:
# Carregar o split "test" da versão multilabel
dataset = load_dataset("Silly-Machine/TuPyE-Dataset", "multilabel", split="test")

# Converter para DataFrame
df_tupye = dataset.to_pandas()

# Usar apenas a coluna de texto, e renomear para "comentario"
dados = df_tupye[["text"]].rename(columns={"text": "comentario"}).copy()


# Mostra os nomes das colunas para conferir
print(df_tupye.columns)

Index(['text', 'aggressive', 'hate', 'ageism', 'aporophobia', 'body_shame',
       'capacitism', 'lgbtphobia', 'political', 'racism',
       'religious_intolerance', 'misogyny', 'xenophobia', 'other',
       'researcher', 'year', 'source'],
      dtype='object')


In [181]:
leet_map = {
    '4': 'a',
    '3': 'e',
    '1': 'i',
    '0': 'o',
    '7': 't',
    '5': 's',
    '$': 's',
    '@': 'a',
    '!': 'i',
    '|_|': 'u',
    '|-|': 'h',
    '\\/': 'v',
    '><': 'x'
}

In [211]:
def leet_to_text(text):
    """Convert leet speak to normal text only within words"""
    words = text.split()
    converted_words = []
    
    for word in words:
        # Se a palavra é composta apenas de dígitos, mantém original
        if word.isdigit():
            converted_words.append(word)
        else:
            # Converte caracteres leet no meio de palavras
            converted_word = []
            for char in word:
                converted_char = leet_map.get(char, char)
                converted_word.append(converted_char)
            converted_words.append(''.join(converted_word))
    
    return ' '.join(converted_words)

def preprocessar_texto(text):
    """Full text preprocessing pipeline"""
    # 1. Remover menções tipo @User
    text = re.sub(r'@[\w]+', '', text)
    
    # 2. Substituir linguagem hacker
    text = leet_to_text(text)
    
    # 3. Lowercase e normalizar acentos
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    
    # 4. Manter apenas letras, números, espaços e emojis
    text = ''.join(c if c.isalnum() or c.isspace() or emoji.is_emoji(c) else ' ' for c in text)
    
    # 5. Remover múltiplos espaços
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [212]:
# ⚠️ Dicionário expandido por categoria
dic_categorias = {
    "ageism": [
        "velho inútil", "caduco", "aposentado folgado", "idoso inútil", "veio chato",
        "velhote", "carcaça velha", "já devia estar morto"
    ],
    "aporophobia": [
        "favelado", "mendigo", "vagabundo", "miserável", "pobre fedido", "lixo humano", "lixo social", 
        "maloqueiro", "pé rapado"
    ],
    "body_shame": [
        "gordo", "obeso", "baleia", "barrigudo", "pançudo", "magrelo", "anorexica", 
        "palito", "frangote", "feio", "horrível", "corpo escroto"
    ],
    "capacitism": [
        "retardado", "mongol", "aleijado", "deficiente inútil", "débil mental", "doente mental", 
        "abobado", "tapado", "lesado", "esquisito", "demente"
    ],
    "lgbtphobia": [
        "viado", "bicha", "sapatão", "traveco", "gay nojento", "mariquinha", "boiola", 
        "pervertido", "viado do caralho", "doença gay", "cura gay"
    ],
    "political": [
        "bolsominion", "petista", "comunista", "esquerdista", "fascista", "vermelho", 
        "nazista", "isentão", "genocida", "pilantra político", "direitopata"
    ],
    "racism": [
        "macaco", "crioulo", "preto imundo", "denegrir", "escravo", "negrada", "quilombola folgado", 
        "raça inferior", "negro sujo", "zumbi fedido", "branquelo escroto"
    ],
    "religious_intolerance": [
        "fanático religioso", "crente burro", "pastor ladrão", "macumbeiro", "satanista", "evanjegue", 
        "padre pedófilo", "intolerante de fé", "seita maldita", "religião do diabo"
    ],
    "misogyny": [
        "vadia", "vagabunda", "puta", "lugar de mulher", "burra", "feminazi", "histérica", 
        "mulher não serve pra nada", "chata demais", "tinha que ser mulher", "mulherzinha"
    ],
    "xenophobia": [
        "estrangeiro nojento", "gringo burro", "chinês maldito", "haitiano lixo", "invasor", 
        "venezuelano safado", "muçulmano terrorista", "cubano comunista", "gringo folgado"
    ],
    "other": [
        "idiota", "otário", "imbecil", "escroto", "nojento", "burro", "desgraçado", "bosta", 
        "merda", "inútil", "babaca", "canalha", "corno", "arrombado"
    ]
}

In [213]:
# Emojis agressivos
emojis_agressivos = {"💢", "😠", "🤬", "😡", "👊", "🖕", "😤", "😒"}

In [224]:
# Modifique onde você processa o DataFrame:
dados['comentario'] = dados['comentario'].apply(preprocessar_texto)

In [225]:
def normalize_keep_emojis(text):
    """Normalize text while preserving emojis (SEM chamar preprocessar_texto novamente)"""
    text = text.lower()
    text_sem_acentos = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    resultado = ''
    for c in text:
        if c in text_sem_acentos or emoji.is_emoji(c):
            resultado += c
        else:
            resultado += ' '
    return resultado

In [226]:
def detectar_categorias(text):
    text_proc = normalize_keep_emojis(text)
    resultado = {}
    for categoria, palavras in dic_categorias.items():
        # Verifica tanto a palavra original quanto sua possível versão em leet
        resultado[categoria] = any(
            p in text_proc or 
            leet_to_text(p) in text_proc  # <-- Adiciona esta verificação
            for p in palavras
        )
    resultado["emoji_agressivo"] = any(e in text_proc for e in emojis_agressivos)
    if resultado["emoji_agressivo"]:
        resultado["Other"] = True
    resultado["hate"] = any(resultado[c] for c in dic_categorias.keys())
    resultado["aggressive"] = resultado["hate"] or resultado["emoji_agressivo"]
    return resultado

In [227]:
# Score e classificação agressividade
def calcular_score(text):
    text_proc = normalize_keep_emojis(text)
    
    # Conta palavras únicas do dicionário encontradas
    palavras_encontradas = set()
    for palavras in dic_categorias.values():
        for p in palavras:
            if p in text_proc:
                palavras_encontradas.add(p)
    score_palavras = len(palavras_encontradas)
    
    # Conta emojis agressivos
    score_emojis = sum(text_proc.count(e) for e in emojis_agressivos)
    
    score = score_palavras + score_emojis
    
    if score == 0:
        label = "Non-aggressive"
    elif score <= 2:
        label = "Aggressive - Not hate"
    else:
        label = "Aggressive - Hate"
    
    return score, label


In [218]:
dados = df_tupye[["text"]].rename(columns={"text": "comentario"}).copy()

In [219]:
# Aplicar multilabel
multilabels = dados["comentario"].apply(detectar_categorias)
df_labels = pd.DataFrame(list(multilabels))
dados = pd.concat([dados, df_labels], axis=1)

In [220]:
# Calcular score e classificação agressividade
dados[["score_agressividade", "classificacao"]] = dados["comentario"].apply(
    lambda x: pd.Series(calcular_score(x))
)

In [238]:
def print_multilabel_metrics(y_true, y_pred):
    f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0)
    f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    hamming = hamming_loss(y_true, y_pred)
    exact = accuracy_score(y_true, y_pred)

    print("\n📊 Avaliação Multilabel")
    print("=" * 30)
    print(f"✔️ F1 Score (Micro):     {f1_micro:.4f}")
    print(f"✔️ F1 Score (Macro):     {f1_macro:.4f}")
    print(f"✔️ F1 Score (Weighted):  {f1_weighted:.4f}")
    print(f"⚠️ Hamming Loss:         {hamming:.4f}")
    print(f"✅ Subset Accuracy:      {exact:.4f}")
    print("=" * 30)

In [228]:
# Exibir resultado
print(dados)

                                             comentario  ageism  aporophobia  \
0     o que dizer da atitude de um ministro cuja pas...   False        False   
1                         fernando haddad me representa   False        False   
2       sou branco mas tenho vergonha da minha raca pqp   False        False   
3                    todo mes isso paga quem quer porra   False        False   
4                                   menina insuportavel   False        False   
...                                                 ...     ...          ...   
8729   rt 45 e o numero de tiros que tu vai levar no cu   False        False   
8730                                  viado com certeza   False        False   
8731                                     fala nao porra   False        False   
8732  paulo freire era tao bom que nenhum pais no mu...   False        False   
8733  vai ficar feio mesmo quando a reforma da previ...   False        False   

      body_shame  capacitism  lgbtphobi

In [158]:
pd.DataFrame(dados)

,comentario,ageism,aporophobia,body_shame,capacitism,lgbtphobia,political,racism,religious_intolerance,misogyny,xenophobia,other,emoji_agressivo,hate,aggressive,Other,score_agressividade,classificacao
0,@user @user O que dizer da atitude de um minis...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,0,Non-aggressive
1,Fernando Haddad me representa,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,0,Non-aggressive
2,sou branco mas tenho vergonha da minha raça pqp,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,0,Non-aggressive
3,@user todo mês isso paga quem quer porra,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,0,Non-aggressive
4,Menina insuportável,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,0,Non-aggressive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8729,rt @user @user 45 é o número de tiros que tu v...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,0,Non-aggressive
8730,@user @user viado com certeza...,False,False,False,False,True,False,False,False,False,False,False,False,True,True,NaN,1,Aggressive - Not hate
8731,@user @user fala não porra 🤦🏾‍♀️,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,0,Non-aggressive
8732,Paulo Freire era tão bom que nenhum país no mu...,False,False,False,False,False,True,False,False,False,False,False,False,True,True,NaN,1,Aggressive - Not hate


In [231]:
target = ['aggressive', 'hate', 'ageism', 'aporophobia', 'body_shame', 'capacitism', 'lgbtphobia', 'political', 'racism', 'religious_intolerance', 'misogyny', 'xenophobia', 'other']

In [232]:
#y_real obtido apartir do valores do dataset
y_real = df_tupye[target].values

In [233]:
#y_pred obrido apartir dos valores calculados pelo algoritmo usando regras
y_pred = pd.DataFrame(dados)

In [234]:
y_pred = y_pred[target].values.astype(int)

In [237]:
print_multilabel_metrics(y_real, y_pred)


📊 Avaliação Multilabel
✔️ F1 Score (Micro):     0.2966
✔️ F1 Score (Macro):     0.1451
✔️ F1 Score (Weighted):  0.2985
⚠️ Hamming Loss:         0.0525
✅ Subset Accuracy:      0.6820
